# Media Campaign Data Cleanup

  Author: Joshua Guillen

___


## Background
___

## Summary
___

## Table of Contents
___

## 0. Setup / Dependencies
---

Importing Google goodness

In [ ]:
!pip install google-cloud
!pip install google-cloud-storage
!pip install google-cloud-bigquery[pandas]

In [ ]:
from google.colab import auth
auth.authenticate_user()

Connecting to my project's data warehouse in BigQuery.

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project='sfda-capstone-project')

Importing Audience Reporting, Device Location Creative Reporting, and Inventory Viewing Data Sets.


In [ ]:
QUERY = """
SELECT *
FROM sfda-capstone-project.datasets.device_location_creative_reporting
"""
results = client.query(QUERY)
device_location = results.to_dataframe()


In [ ]:
QUERY = """
SELECT *
FROM sfda-capstone-project.datasets.inventory_viewability_reporting
"""
results = client.query(QUERY)
inventory_viewability = results.to_dataframe()

## 1. Device Location Creative Reporting
___

### Initialization

Making copy of raw query for cleaning.

In [ ]:
device_location_clean = device_location.copy()

In [ ]:
device_location_clean.head()

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Device_Model,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,None,None,None,None,None,None,None,None,<NA>,<NA>,<NA>,NaN,NaN
1,Mobile_2P_Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000974,0.000313
2,Mobile_2P_In Market_Business Services or ISPs,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000583,0.000187
3,Mobile_2P_Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.000662,0.000213
4,Mobile_2P_Custom Intent/Affinity,"Chicago, Illinois",Google Chrome,Android 8.1,320x50,Smart Phone,BQ (All Models),Unknown,1,0,0,0.001402,0.000450


Removing Nulls on 0th Row

In [ ]:
device_location_clean = device_location_clean.drop(0).reset_index(drop=True)

Checking for NULL values

In [ ]:
device_location_clean.isna().sum()

Strategy             0
DMA_Name             0
Browser              0
Operating_System     0
Creative_Size        0
Device_Type          0
Device_Make          0
Device_Model         0
Impressions          0
Clicks               0
Total_Conversions    0
Gross_Cost           0
Net_Cost             0
dtype: int64

Dataframe Info

In [ ]:
device_location_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464433 entries, 0 to 464432
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Strategy           464433 non-null  object 
 1   DMA_Name           464433 non-null  object 
 2   Browser            464433 non-null  object 
 3   Operating_System   464433 non-null  object 
 4   Creative_Size      464433 non-null  object 
 5   Device_Type        464433 non-null  object 
 6   Device_Make        464433 non-null  object 
 7   Device_Model       464433 non-null  object 
 8   Impressions        464433 non-null  Int64  
 9   Clicks             464433 non-null  Int64  
 10  Total_Conversions  464433 non-null  Int64  
 11  Gross_Cost         464433 non-null  float64
 12  Net_Cost           464433 non-null  float64
dtypes: Int64(3), float64(2), object(8)
memory usage: 47.4+ MB


### Unique Values

Looking at unique values in Strategy category. It seems every audience is a 2nd party mobile custom audience, except for RON, which means 'Run of Network', which we could categorize as a custom audience. These labels would benefit from some better formatting such as removing the 'Mobile 2P' and the underscores.

In [ ]:
device_location_clean.Strategy.unique()

array(['Mobile_2P_Custom Intent/Affinity',
       'Mobile_2P_In Market_Business Services or ISPs',
       'Mobile_2P_Business Contextual', 'Mobile_RON',
       'Mobile_2P_Pixel Converters_Optimized Targeting',
       'Mobile_2P_Affinity Categories'], dtype=object)

Looking at unique values in Device Model. Note: One of the common values is 'Unknown' which requires some more exploration to determine what to do that data.

In [ ]:
device_location_clean.Device_Model.unique()

array(['Unknown', 'bq AQUARIS X', 'LG V60 ThinQ 5G (LM-V600)', ...,
       'General Mobile GM 5 PLUS D', 'General Mobile GM8 GO',
       'General Mobile GM 8 D'], dtype=object)

Unique values in device type.

In [ ]:
device_location_clean.Device_Type.unique()

array(['Smart Phone', 'Tablet'], dtype=object)

Unique values in Browser

In [ ]:
device_location_clean.iloc[:, 2].unique()

array(['Google Chrome', 'Yandex', 'Microsoft Edge', 'Android Browser'],
      dtype=object)

Unique values in OS

In [ ]:
device_location_clean.iloc[:, 3].unique()

array(['Android 4.4', 'Android 8.1', 'Android 10.0', 'Android 11.0',
       'Android 4.2', 'Android 5.0', 'Android 9.0', 'WindowsPhone 10.0',
       'Android', 'Android 5.1', 'Android 6.0', 'Android 7.0',
       'Android 7.1', 'Android 8.0', 'Android 12.0', 'Android 13.0',
       'Android 4.3', 'iOS', 'iOS 7.0', 'iOS 9.0', 'iOS 9.2', 'iOS 10.0',
       'iOS 10.1', 'iOS 10.2', 'iOS 10.3', 'iOS 11.0', 'iOS 11.1',
       'iOS 11.2', 'iOS 11.3', 'iOS 11.4', 'iOS 12.0', 'iOS 12.1',
       'iOS 12.2', 'iOS 12.3', 'iOS 12.4', 'iOS 12.5', 'iOS 13.0',
       'iOS 13.1', 'iOS 13.2', 'iOS 13.3', 'iOS 13.4', 'iOS 13.5',
       'iOS 13.6', 'iOS 13.7', 'iOS 14.0', 'iOS 14.1', 'iOS 14.2',
       'iOS 14.3', 'iOS 14.4', 'iOS 14.5', 'iOS 14.6', 'iOS 14.7',
       'iOS 14.8', 'iOS 15.0', 'iOS 15.1', 'iOS 15.2', 'iOS 15.3',
       'iOS 15.4', 'iOS 15.5', 'iOS 15.6', 'iOS 15.7', 'iOS 16.0',
       'iOS 16.1', 'iOS 16.2', 'iOS 16.3', 'iOS 16.4', 'iOS 16.5',
       'Android 4.0', 'Android 4.1', 'Android 2.3

### Exploring Unknown Values

Filtering for unknown devices

In [ ]:
unknown_devices_filter = device_location_clean.loc[:, 'Device_Model'] == "Unknown"

In [ ]:
unknown_devices = device_location_clean.loc[unknown_devices_filter, :]

Over 67k unknown entries!

In [ ]:
unknown_devices.shape

(67488, 13)

There are certain Device Makes that do not have unknown values.

In [ ]:
list(unknown_devices.Device_Make.unique()) == list(device_location_clean.Device_Make.unique())

False

In [ ]:
unknown_devices.Device_Make.unique()

array(['BQ (All Models)', 'HP (All Models)', 'LG (All Models)',
       'Blu (All Models)', 'HTC (All Models)', 'IMO (All Models)',
       'NEC (All Models)', 'NGM (All Models)', 'RCA (All Models)',
       'TCL (All Models)', 'THL (All Models)', 'ZTE (All Models)',
       'Acer (All Models)', 'Asus (All Models)', 'Dell (All Models)',
       'Eton (All Models)', 'LAVA (All Models)', 'OPPO (All Models)',
       'Palm (All Models)', 'Sony (All Models)', 'Vivo (All Models)',
       'Wiko (All Models)', 'Advan (All Models)', 'Azumi (All Models)',
       'Chuwi (All Models)', 'Cubot (All Models)', 'Haier (All Models)',
       'Intel (All Models)', 'Intex (All Models)', 'Lanix (All Models)',
       'Meizu (All Models)', 'Nokia (All Models)', 'Sharp (All Models)',
       'Sonim (All Models)', 'Tecno (All Models)', 'Vizio (All Models)',
       'iBall (All Models)', 'Amazon (All Models)', 'Archos (All Models)',
       'Doogee (All Models)', 'Explay (All Models)',
       'Gionee (All Models)', 'Go

Only 8 makes have no unknown models in their rows.

In [ ]:
unknown_makes = list(unknown_devices.Device_Make.unique())
all_makes = list(device_location_clean.Device_Make.unique())
not_in_unknown = []

# find the Device Makes that do not have any unknown models
for i in range(len(all_makes)):
    if (all_makes[i] not in unknown_makes):
        not_in_unknown.append(all_makes[i])

not_in_unknown

['HDC (All Models)',
 'Apple (All Models)',
 'Itoos (All Models)',
 'DoCoMo (All Models)',
 'Yifang (All Models)',
 'Pioneer (All Models)',
 'Vodafone (All Models)',
 'SonyEricsson (All Models)']

Since the Device Make column has a significant amount of unknown values (67,488), all of which can not be filled in with existing data, it should be dropped entirely since any analysis of the Device Makes would be skewed and not representative of the true audience.

In [ ]:
device_location_clean = device_location_clean.drop('Device_Model', axis=1)
device_location_clean.head()

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,Mobile_2P_Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000974,0.000313
1,Mobile_2P_In Market_Business Services or ISPs,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000583,0.000187
2,Mobile_2P_Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),1,0,0,0.000662,0.000213
3,Mobile_2P_Custom Intent/Affinity,"Chicago, Illinois",Google Chrome,Android 8.1,320x50,Smart Phone,BQ (All Models),1,0,0,0.001402,0.000450
4,Mobile_2P_Custom Intent/Affinity,"Detroit, Michigan",Google Chrome,Android 8.1,300x50,Smart Phone,BQ (All Models),1,0,0,0.000703,0.000226


### Reformatting Strategy Column

The "Mobile" and "2P" labels in the Strategy are redundant since it applies to every entry. Therefore we will update every value under this category with only their specific strategy.

In [ ]:
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Custom Intent/Affinity', value='Custom Intent/Affinity')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_In Market_Business Services or ISPs', value='In Market, Business Services or ISPs')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Business Contextual', value='Business Contextual')
device_location_clean = device_location_clean.replace(to_replace='Mobile_RON', value='Run of Network')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Pixel Converters_Optimized Targeting', value='Pixel Converters, Optimized Targeting')
device_location_clean = device_location_clean.replace(to_replace='Mobile_2P_Affinity Categories', value='Affinity Categories')

If there were more than 6 distinct values, I would have considered a creating a more sophisticated method of updating these values. This is fine for now.

In [ ]:
device_location_clean.head(3)

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost
0,Custom Intent/Affinity,"Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000974,0.000313
1,"In Market, Business Services or ISPs","Miami-Ft. Lauderdale, Florida",Google Chrome,Android 4.4,300x250,Smart Phone,BQ (All Models),1,0,0,0.000583,0.000187
2,Business Contextual,"Miami-Ft. Lauderdale, Florida",Yandex,Android 8.1,320x50,Smart Phone,BQ (All Models),1,0,0,0.000662,0.000213


### Adding Conversion Value

I can create a new column, "Conversions_Revenue" by multiplying each row's total conversions by the conversion value. From our [guidelines](https://docs.google.com/presentation/d/1zieAer3-1WF3rS6wtzrh0eO-KtSYHl0JVq19Q-iwp-Y/edit#slide=id.g24db77693a5_0_457), the conversion value for a custom audience is $69.45.

In [ ]:
# df.loc[:, 'new_column'] = round(df['col1'] / df['col2'])
device_location_clean.loc[:, 'Conversions_Revenue'] = device_location_clean.Total_Conversions * 69.45

Double-checking to see if it worked shows our top Conversion revenue as $2,222.40 from 32 total conversions.

In [ ]:
device_location_clean.sort_values('Conversions_Revenue', ascending=False).head(3)

,Strategy,DMA_Name,Browser,Operating_System,Creative_Size,Device_Type,Device_Make,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Conversions_Revenue
57455,"Pixel Converters, Optimized Targeting","Grand Rapids-Kalamazoo, Michigan",Google Chrome,Android 10.0,320x50,Smart Phone,Wiko (All Models),89539,24,32,149.118518,47.867044,2222.4
57434,"Pixel Converters, Optimized Targeting","Chicago, Illinois",Google Chrome,Android 10.0,320x50,Smart Phone,Wiko (All Models),558213,246,28,1065.355671,341.979170,1944.6
83888,"Pixel Converters, Optimized Targeting","Atlanta, Georgia",Google Chrome,iOS 16.5,300x250,Smart Phone,Apple (All Models),168619,47,25,270.247682,86.749506,1736.25


### Exporting clean CSV

Our Device Location dataset is satisfactorily clean for my liking. Time to export it!

In [ ]:
device_location_clean.to_csv('device_location_clean.csv')

## 2. Inventory Viewability Reporting
___

Initial View

In [ ]:
inventory_viewability.head()

,Brand_Company_Name,Strategy__,App_URL,Exchange,Impressions,Clicks,Total_Conversions,Gross_Cost,Net_Cost,Viewable_Impressions,Measurable_Impressions
0,Choose your advertiser name,Desktop_2P_Affinity Categories,al.com,Taboola,2401,0,0,3.106056,0.997044,1551,2122
1,Choose your advertiser name,Desktop_2P_Affinity Categories,carscoops.com,Taboola,148,0,0,0.527778,0.169417,37,126
2,Choose your advertiser name,Desktop_2P_Affinity Categories,cleveland.com,Taboola,180,0,0,0.235679,0.075653,97,151
3,Choose your advertiser name,Desktop_2P_Affinity Categories,doviz.com,Adform,19,0,0,0.003110,0.000998,12,21
4,Choose your advertiser name,Desktop_2P_Affinity Categories,e.walla.co.il,Taboola,26,0,0,0.032122,0.010311,7,27


Data has over a million rows (1,037,468) across 11 rows!

In [ ]:
inventory_viewability.shape

(1037468, 11)

Inventory Viewability dataset has financial information on cost

In [ ]:
inventory_viewability.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037468 entries, 0 to 1037467
Data columns (total 11 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Brand_Company_Name      1037466 non-null  object 
 1   Strategy__              1037466 non-null  object 
 2   App_URL                 1037466 non-null  object 
 3   Exchange                1037466 non-null  object 
 4   Impressions             1037466 non-null  Int64  
 5   Clicks                  1037466 non-null  Int64  
 6   Total_Conversions       1037466 non-null  Int64  
 7   Gross_Cost              1037466 non-null  float64
 8   Net_Cost                1037466 non-null  float64
 9   Viewable_Impressions    1037466 non-null  Int64  
 10  Measurable_Impressions  1037466 non-null  Int64  
dtypes: Int64(5), float64(2), object(4)
memory usage: 92.0+ MB
